# Algorithme de Grover

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def grover_image(show_oracle=False, show_grover=False):

    fig, ax = plt.subplots(figsize=(5, 5))

    # Axes
    ax.arrow(0, 0, 1.2, 0, head_width=0.04, head_length=0.06, fc='black', ec='black', linewidth=1)
    ax.arrow(0, 0, 0, 1.2, head_width=0.04, head_length=0.06, fc='black', ec='black', linewidth=1)
    ax.text(1.3, 0, r'$|\text{états}\perp x_s\rangle$', fontsize=12, ha='left')
    ax.text(-0.05, 1.25, r'$|x_s\rangle$', fontsize=12, ha='right')

    # Initial state
    r = 1.0
    theta_0 = np.pi / 18
    x_init = r * np.cos(theta_0)
    y_init = r * np.sin(theta_0)
    ax.arrow(0, 0, x_init, y_init, head_width=0.04, head_length=0.06, 
            fc='blue', ec='blue', linewidth=1)
    ax.text(x_init+0.1, y_init, r'$|+\rangle$', fontsize=12, ha='left')

    # Oracle
    if (show_oracle):
        x_oracle = x_init
        y_oracle = -y_init
        ax.arrow(0, 0, x_oracle, y_oracle, head_width=0.04, head_length=0.06, 
                fc='white', ec='blue', linewidth=1, linestyle='-')
        ax.text(x_init+0.1, -y_init, r'$U_f\,|+\rangle$', fontsize=12, ha='left')

    # Diffusion operator
    if (show_grover):
        theta_new = 3 * theta_0
        x_new = r * np.cos(theta_new)
        y_new = r * np.sin(theta_new)
        ax.arrow(0, 0, x_new, y_new, head_width=0.04, head_length=0.06, fc='green', ec='green', linewidth=1)
        ax.text(x_new+0.1, y_new, r'$G\,|+\rangle$', fontsize=12, ha='left')

    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])

    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.show()

Problème de recherche, exemple:
- identifier une entrée dans une dase de données non-structurée
- identifier la solution d'un problème NP-complexe


Solutions possibles $x\in\{0,1\}^n$ (encodées sur $n$ bits)

On dispose d'une fonction $f$ permettant d'identifier la solution si celle-ci est donnée:

- $f(x) = 1$ si $x$ est solution
- $f(x) = 0$ sinon

On appelle une telle fonction un "oracle"

On se sert de l'oracle pour construire une porte quantique que l'on peut insérer dans un circuit:

\begin{equation*}
    U_f \ket{x} = (-1)^{f(x)} \ket{x}
\end{equation*}

Cette porte logique "reconnaît" la solution du problème décrit par $f$ et a pour effet d'inverser le signe de la composante vectorielle dans la direction de la solution.

En pratique, une telle porte logique est implémentée en utilisant un qubit supplémentaire (*ancilla*), que nous pouvons changer de signe si l'oracle reconnaît $x$ comme solution: $(f(x)=1)$. Cette opération est souvent écrite en utilisant une opération XOR: 

\begin{equation*}
    \tilde U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus f(x)}
\end{equation*}

En initialisant le qubit $\ket y$ avec l'état $\ket- = \frac{1}{\sqrt 2}(\ket 0 - \ket 1)$, celui-ci reste inchangé par l'action de $\tilde U_f$ et nous construisons bien une porte équivalente à $U_f$,

\begin{equation*}
    \tilde U_f \ket{x}\ket{-} = (-1)^{f(x)} \ket{x}\ket{-}
\end{equation*}

En général, la discussion de ce genre d'algorithme se soucie peu de l'implémentation de l'oracle. Toutefois, comme le modèle de calcul quantique généralise le modèle classique, il est théoriquement possible d'implémenter le calcul de $f(x)$.

L'atout du modèle de calcul quantique se trouve dans la possibilité d'évaluer en parallèle toutes les valeurs possibles de $x$ qui peuvent être encodées dans l'état $\ket x$. Cela est réalisé en initialisant les qubits dans l'état $\ket +$ par l'application d'une porte de Hadamard avant l'oracle $U_f$.

Nous pouvons représenter l'action de l'oracle géométriquement sur un graphe où l'on place l'état solution $\ket{x_s}$ (inconnu) sur l'axe $y$ et l'espace de tous les autres états orthogonaux sur l'axe $x$. L'état initial $\ket{+}$ se retrouve à priori avec des composantes arbitraires dans chaque direction, y compris une petite composante dans la direction de la solution.  

In [ ]:
grover_image(show_oracle=True)

Pour exploiter l'action de l'oracle et rapprocher notre état quantique de la solution $\ket{x_s}$, nous réalisons ensuite une opération géométrique de réflexion autour de l'état initial $\ket{+}$. Cette opération est implémentée par l'opérateur

\begin{equation}
    2|+\rangle \langle+| - I
\end{equation}

**Expliquer pourquoi cela réalise une réflexion: opérateur projection $|+\rangle \langle+|$, etc.**

Nous représenterons une itération complète de l'algorithme de Grover par l'opérateur $G$, qui résulte de l'application successive de l'oracle $U_f$ et de la réflexion ci-dessus.

In [ ]:
grover_image(show_oracle=True, show_grover=True)

Une itération de Grover a pour effet de rapprocher l'état actuel $\ket{\psi}$ de l'état solution $\ket{x_s}$ (la composante vectorielle dans la direction de la solution est amplifiée par l'algorithme).

Initialement, la composante de l'état dans la direction de la solution $\ket{x_s}$ a une amplitude arbitraire pas nécessairement plus grande que dans les $n-1$ autres directions. A ce stade, la probabilité d'observer l'état quantique dans l'état solution $\ket{x_s}$ est de $O(1/n)$. Le vecteur $\ket{\psi}$ est à peine incliné le long de la direction solution, avec un angle $\theta = O(1/\sqrt{n})$. Chaque itération de Grover rapproche l'état quantique de la solution d'un angle $2\theta$. Il faudra donc $O(\sqrt{n})$ itérations de Grover pour avoir une probabilité suffisamment grande de mesurer notre état quantique dans l'état $\ket{x_s}$. 

Attention: une fois le vecteur suffisamment rapproché de la solution $\ket{x_s}$, l'itération de Grover cesse d'amplifier la composante dans la direction de la solution. La probabilité exacte de mesurer notre état quantique dans l'état solution après $k$ itérations de Grover est

\begin{equation*}
    |\braket{x_s|\psi}|^2 = \sin^2((2k+1)\theta)
\end{equation*}

La probabilité est maximale pour $k \approx \pi/4\theta$ itérations, mais comme la valeur de $\theta$ est inconnue, nous ne pouvons pas garantir la mesure de la solution à chaque exécution de l'algorithme. Il est donc nécessaire de répéter l'algorithme de Grover un nombre indéterminé mais $O(1)$ fois pour obtenir la solution de notre problème. Cela ne change cependant pas la complexité $O(\sqrt{n})$ de l'algorithme et l'avantage que présente celui-ci sur les méthodes classiques.

![title](../figures/grover_full.png)

![title](../figures/grover_iter.png)

## Oracle qui identifie un état particulier

In [ ]:
from qiskit.quantum_info import Operator
from qiskit import QuantumCircuit
import numpy as np

def oracle(bitstring):

    n = len(bitstring)
    oracle_matrix = np.eye(2**n)
    index = int(bitstring, 2)
    oracle_matrix[index, index] = -1
    return Operator(oracle_matrix)

In [ ]:
etat_reconnu = "0010"

Uf = oracle(etat_reconnu)
display(Uf.draw("latex"))

n = len(etat_reconnu)
circuit_oracle = QuantumCircuit(n)
circuit_oracle.unitary(Uf, circuit_oracle.qubits, label='Uf')
circuit_oracle.draw('mpl')

## Circuit avec itérations de Grover

In [ ]:
from qiskit.circuit.library import grover_operator

circuit_grover_iter = grover_operator(circuit_oracle)
circuit_grover_iter.draw('mpl')
#circuit_grover_iter.decompose().draw('mpl')

In [ ]:
circuit = QuantumCircuit(n)

# Superposition initiale
circuit.h(range(n))

# Iterations de Grover
#num_iters = 1
num_iters = np.floor(np.sqrt(2**n))

for _ in range(int(num_iters)):
    circuit.append(grover_operator(circuit_oracle).to_gate(), range(n))

circuit.measure_all()
circuit.draw('mpl')

In [ ]:
### Too much noise: one grover iteration is better in this case
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2
#backend = FakeManilaV2()

from qiskit_aer import AerSimulator
backend = AerSimulator()

In [ ]:
from qiskit.compiler import transpile

circuit_transpiled = transpile(circuit, backend, optimization_level=3)
#circuit_transpiled.draw('mpl')

In [ ]:
job = backend.run(circuit_transpiled, shots=1024)

result = job.result()

from qiskit.visualization import plot_histogram
plot_histogram(result.get_counts())